In [19]:
# 전쳐리
# db 

In [36]:
# 전처리 
import pandas as pd 
import numpy as np
from datetime import datetime, timedelta
roadkill_df = pd.read_csv('한국도로공사_로드킬 데이터 정보_20250501.csv', encoding='cp949')
roadkill_df.columns = (
    roadkill_df.columns
    .str.replace('\ufeff', '', regex=True)
    .str.strip()
)
roadkill_df.head()
roadkill_df.info()
print(roadkill_df.columns.tolist())
# 함수: timestamp, status 
def add_timestamp(df: pd.DataFrame,
                  start_dt: str | None = None,
                  step_seconds: int = 240,
                  as_string: bool = True,
                  **kwargs) -> pd.DataFrame:
    # return_string 별칭 허용
    if 'return_string' in kwargs:
        as_string = kwargs.pop('return_string')

    out = df.copy()
    base = datetime.now().replace(microsecond=0) if start_dt is None else pd.to_datetime(start_dt).to_pydatetime()
    times = [base + timedelta(seconds=i * step_seconds) for i in range(len(out))]
    if as_string:
        out['timestamp'] = pd.to_datetime(times).strftime('%Y-%m-%d %H:%M:%S')
    else:
        out['timestamp'] = pd.to_datetime(times)
    return out
def apply_status_rules_dups_only(
    df: pd.DataFrame,
    group_cols=('위도','경도'),
    time_col='timestamp',
    status_col='status',
    keep_order=True
) -> pd.DataFrame:
    """
    규칙(중복 좌표에만 적용):
      1) 첫 이벤트 = 0
      2) 첫 이벤트 시각으로부터 ≤ 1시간 내 나중 이벤트 = 1
      3) '1'이 찍힌 시각으로부터 ≥ 24시간 후 나중 이벤트 = 2
    단일 좌표(해당 좌표가 1건뿐)는 status 변경 없음.
    """
    out = df.copy()
    orig_idx = out.index
    out[time_col] = pd.to_datetime(out[time_col], errors='coerce')

    # 중복 좌표만 추려서 처리
    sizes = out.groupby(list(group_cols), sort=False)[time_col].transform('size')
    dup_mask = sizes >= 2
    work = out.loc[dup_mask].sort_values(list(group_cols) + [time_col]).copy()
    if work.empty:
        return out  # 중복 좌표가 없다면 원본 그대로

    H1  = pd.Timedelta(hours=1).value      # ns
    H24 = pd.Timedelta(hours=24).value     # ns

    def assign(g: pd.DataFrame) -> pd.DataFrame:
        ts = g[time_col].to_numpy('datetime64[ns]')
        st = np.zeros(len(g), dtype=np.int8)  # 기본 0 (첫 이벤트 0)

        if len(g) >= 2:
            # 첫 이벤트 기준 경과 시간(ns)
            d = (ts - ts[0]).astype('timedelta64[ns]').astype('int64')

            # 1시간 이내(>0, ≤1h) → 1
            within_1h = (d > 0) & (d <= H1)
            st[within_1h] = 1

            # 첫 '1' 이후 24시간 지난 시점부터 → 2
            idx1 = np.flatnonzero(within_1h)
            if idx1.size > 0:
                i1 = idx1[0]
                d2 = (ts - ts[i1]).astype('timedelta64[ns]').astype('int64')
                i2 = np.searchsorted(d2, H24, side='left')
                if i2 < len(g):
                    st[i2:] = 2

        g[status_col] = st
        return g

    work = work.groupby(list(group_cols), group_keys=False, sort=False).apply(assign)
    out.loc[work.index, status_col] = work[status_col].astype('int8')

    return out.loc[orig_idx] if keep_order else out
roadkill_info2  = roadkill_df[['본부명','지사명', '노선명', '방 향', '발생건수', '위도', '경도']].copy()
# status 랜덤으로 주기 
np.random.seed(42)
# 0=사망, 1=발견, 2=재발견 (동일 확률)
roadkill_info2['status'] = np.random.choice([0, 1, 2], size=len(roadkill_info2)).astype('int8')

# insert되는 timestamp추가 (일단은 임의의 값)
roadkill_info2 = add_timestamp(roadkill_info2, step_seconds=240, return_string=True)

# 상태 변경하기 
roadkill_info2 = apply_status_rules_dups_only(roadkill_info2)

roadkill_info2['추정치'] = ''

roadkill_info2.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59 entries, 0 to 58
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   본부명     59 non-null     object 
 1   지사명     59 non-null     object 
 2   노선명     59 non-null     object 
 3   구간      59 non-null     object 
 4   방 향     59 non-null     object 
 5   5km     59 non-null     int64  
 6   발생건수    59 non-null     int64  
 7   위도      59 non-null     float64
 8   경도      59 non-null     float64
dtypes: float64(2), int64(2), object(5)
memory usage: 4.3+ KB


In [42]:
# 컬럼이름 맞추기 

roadkill_info2.columns = ['head','branch', 'line', 'direction', 'freq', 'lat', 'lon', 'status', 'time','추정치']


In [43]:
roadkill_info2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59 entries, 0 to 58
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   head       59 non-null     object        
 1   branch     59 non-null     object        
 2   line       59 non-null     object        
 3   direction  59 non-null     object        
 4   freq       59 non-null     int64         
 5   lat        59 non-null     float64       
 6   lon        59 non-null     float64       
 7   status     59 non-null     int8          
 8   time       59 non-null     datetime64[ns]
 9   추정치        59 non-null     object        
dtypes: datetime64[ns](1), float64(2), int64(1), int8(1), object(5)
memory usage: 4.7+ KB


In [44]:
# 중복데이터 확인 
# 완전 일치 기준 중복 여부
mask_exact = roadkill_info2.duplicated(subset=['lat','lon'], keep=False)
dups_exact = (roadkill_info2[mask_exact]
              .sort_values(['lat','lon','time']))
print("완전 일치 중복 샘플:")
print(dups_exact.head(10))

완전 일치 중복 샘플:
    head branch         line direction  freq        lat         lon  status  \
9   광주전남     순천          남해선        순천     3  34.990320  127.528375       0   
10  광주전남     순천          남해선        부산     3  34.990320  127.528375       1   
16    전북     부안         서해안선        목포     4  35.357142  126.583441       0   
17    전북     부안         서해안선        목포     3  35.357142  126.583441       1   
45    충북     진천          중부선        남이     5  36.708802  127.437539       0   
46    충북     진천          중부선        하남     3  36.708802  127.437539       1   
37  대전충남     공주  서산영덕선(대전당진)        대전     3  36.782578  126.689747       0   
41    충북     보은  서산영덕선(청주상주)        영덕     3  36.782578  126.689747       1   
51    충북     진천          중부선        남이     3  36.947610  127.471406       0   
52    충북     진천          중부선        하남     3  36.947610  127.471406       1   

                  time 추정치  
9  2025-08-26 13:06:05      
10 2025-08-26 13:10:05      
16 2025-08-26 13:34:05      
1

In [45]:
# 이걸 쿼리에 올리기 

In [48]:
# roadkill_loc (위도+경도 PK), roadkill_freq (지사명 PK) 생성 및 적재

import pandas as pd
import pymysql
from sqlalchemy import create_engine, text

# 1. MySQL 연결 
db_config = {
    'host': 'localhost',
    'user': 'root',
    'passwd': '1234',
    'db': 'roadkill_db',
    'charset': 'utf8'
}

# sqlalchemy engine 
engine = create_engine(
    f"mysql+pymysql://{db_config['user']}:{db_config['passwd']}@{db_config['host']}/{db_config['db']}?charset={db_config['charset']}"
)


# 3. DB테이블 생성
# with engine.begin() as conn:
#     conn.execute(text("DROP TABLE IF EXISTS roadkill_loc"))
#     conn.execute(text("""
#         CREATE TABLE roadkill_info (
#             head       VARCHAR(100) NOT NULL,
#             branch       VARCHAR(100) NOT NULL,
#             line       VARCHAR(120) NOT NULL,
#             direction        VARCHAR(50)  NOT NULL,
#             freq      INT UNSIGNED NOT NULL,
#             lat         DOUBLE NOT NULL,
#             lon         DOUBLE NOT NULL,
#             status      TINYINT NOT NULL COMMENT '0=발견,1=재발견,2=죽음',
#             time        DATETIME NOT NULL,
#             추정치       VARCHAR(200) NULL,
#             PRIMARY KEY (lat, lon, status, time),
#             INDEX idx_ts (time),
#             INDEX idx_branch (head),
#             INDEX idx_line (line)
#         ) CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;
#     """))
        
# 4) 데이터 적재
roadkill_info2.to_sql("roadkill_info", con=engine, if_exists="append", index=False)

print("roadkill_info 적재 완료")

roadkill_info 적재 완료


In [49]:
# branch별 
sql = """
SELECT branch, 
        DATE(time) as dt,
        SUM(freq) as total_freq
FROM roadkill_info
GROUP BY branch, DATE(time)
ORDER BY branch
"""

df_branch_daily = pd.read_sql(sql, con=engine)
print(df_branch_daily)

# with engine.begin() as conn:
#      conn.execute(text("DROP TABLE IF EXISTS roadkill_branch_daily"))
#      conn.execute(text("""
#          CREATE TABLE roadkill_branch_daily (
#              branch      VARCHAR(100) NOT NULL,
#              dt          DATE NOT NULL,
#              total_freq  INT UNSIGNED NOT NULL,
#              PRIMARY KEY (branch, dt)
#          ) CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;
#      """))
df_branch_daily.to_sql('roadkill_branch_daily', con=engine, if_exists='append', index=False)
print("roadkill_branch_daily 적재 완료")

     branch          dt  total_freq
0        강릉  2025-08-26         6.0
1      경기광주  2025-08-26         6.0
2        공주  2025-08-26         9.0
3        남원  2025-08-26        11.0
4        논산  2025-08-26         3.0
5        당진  2025-08-26         3.0
6     당진/공주  2025-08-26         7.0
7        대전  2025-08-26        20.0
8       동서울  2025-08-26         4.0
9        보령  2025-08-26         3.0
10       보은  2025-08-26         3.0
11       부안  2025-08-26        13.0
12       순천  2025-08-26         6.0
13       영동  2025-08-26         3.0
14       원주  2025-08-26         6.0
15       제천  2025-08-26        12.0
16    제천/홍천  2025-08-26         3.0
17       진천  2025-08-26        57.0
18  진천/경기광주  2025-08-26         3.0
19       창녕  2025-08-26         3.0
20       창원  2025-08-26         5.0
21       천안  2025-08-26         6.0
22    천안/순천  2025-08-26         6.0
23       함평  2025-08-26         6.0
24       홍천  2025-08-26         7.0
roadkill_branch_daily 적재 완료
